### Example

In [8]:
#
##
### LITTLE HELPERS

import pandas as pd
import numpy as np
from itertools import product

def _optimize_numeric_dtypes(df):
    import pandas as pd
    float_cols = df.select_dtypes("float").columns
    int_cols = df.select_dtypes("integer").columns
    df[float_cols] = df[float_cols].\
        apply(pd.to_numeric, downcast="float")
    df[int_cols] = df[int_cols].\
        apply(pd.to_numeric, downcast="integer")
    return df

def get_data(path="../data/customer_model/retailrocket/"):

    data = _optimize_numeric_dtypes(
        pd.read_parquet(path))

    data = data.loc[:,["user_id","target_event", "target_customer_value_lag1", "target_customer_value"]]
    return data

def get_acp_stats(data, config):
    gamma = config["gamma"]
    delta = config["delta"]
    psi = config["psi"]
    n_iter = config["n_iter"]
    np.random.seed(config["seed"])

    n_users = data.user_id.nunique()
    sp = []
    for i in range(n_iter):
        gamma_psi = pd.DataFrame.from_dict({
            "user_id":data.user_id.unique(),
            "gamma":np.random.beta(gamma["alpha"], gamma["beta"], size=n_users),
            "psi":np.random.beta(psi["alpha"], psi["beta"], size=n_users)})
        temp = data.merge(gamma_psi, on=["user_id"])
        #temp["ecp"] = (temp["y_pred_proba"]*temp["gamma"]*(temp["target_customer_value_lag1"]-delta)
        #    + (1-temp["y_pred_proba"])*(-temp["psi"]*delta))
        temp["acp"] = (temp["target_event"]*temp["gamma"]*(temp["target_customer_value"]-delta)
            + (1-temp["target_event"])*(-temp["psi"]*delta))
        #sp.append(temp.sort_values("acp", ascending=False).acp.cumsum().max())
        sp.append(temp["acp"].mean())
    res = pd.Series(sp).describe(percentiles=[0.025, 0.975])[["mean", "2.5%", "97.5%"]]
    res["ga"] = gamma["alpha"]
    res["gb"] = gamma["beta"]
    res["gmu"] = gamma["mu"]
    res["gvar"] = gamma["var"]
    res["pa"] = psi["alpha"]
    res["pb"] = psi["beta"]
    res["delta"] = delta
    return res

def get_alpha_beta(mu, var):
    alpha = (-mu**3+mu**2-mu*var)/var
    beta = alpha*(1-mu)/mu
    return (alpha, beta)

def adjust_config(config, mu, var):
    config = config.copy()
    alpha, beta = get_alpha_beta(mu=mu,
        var=var)
    config["gamma"] = {"alpha":alpha, "beta":beta,
        "mu":mu, "var":var}
    return config    

In [9]:
#
##
### ARE WE LIVIN IN A SIM?

mu = [0.001, 0.01, 0.1]
ln_var = [-7.645, -9.939, -12.921]

config_rr = {
    "gamma":{"alpha":2.04, "beta":202.04},
    "delta":7000, 
    "psi":{"alpha":6.12, "beta":3.15},
    "n_iter":1000,
    "seed":1}   

config_rees46 = {
    "gamma":{"alpha":2.04, "beta":202.04},
    "delta":10, 
    "psi":{"alpha":6.12, "beta":3.15},
    "n_iter":1000,
    "seed":1}       

configs = [adjust_config(config_rr, mu, var)
    for mu, var in product(mu, np.exp(ln_var))]

data = get_data()
sim_stats = []
for con in configs:
    sim_stats.append(get_acp_stats(data, con))
sim_stats = pd.concat(sim_stats, axis=1).T
sim_stats["glnvar"] = np.log(sim_stats.gvar)    

In [12]:
sim_stats

,mean,2.5%,97.5%,ga,gb,gmu,gvar,pa,pb,delta,glnvar
0,-459.750116,-488.083831,-375.087990,0.001088,1.086990,0.001,0.000478,6.12,3.15,7000.0,-7.645
1,-457.710524,-481.399424,-419.648583,0.019702,19.682586,0.001,0.000048,6.12,3.15,7000.0,-9.939
2,-457.843241,-474.927895,-440.761704,0.407399,406.991431,0.001,0.000002,6.12,3.15,7000.0,-12.921
3,-318.447016,-388.835235,-223.993364,0.196927,19.495742,0.010,0.000478,6.12,3.15,7000.0,-7.645
4,-316.378460,-347.939576,-283.743712,2.041578,202.116230,0.010,0.000048,6.12,3.15,7000.0,-9.939
5,-317.058075,-333.717021,-301.196279,40.461956,4005.733653,0.010,0.000002,6.12,3.15,7000.0,-12.921
6,1094.019244,1002.429930,1181.413480,18.711517,168.403655,0.100,0.000478,6.12,3.15,7000.0,-7.645
7,1092.967412,1063.021950,1123.910565,186.407098,1677.663884,0.100,0.000048,6.12,3.15,7000.0,-9.939
8,1093.583721,1077.827017,1108.534535,3679.168736,33112.518623,0.100,0.000002,6.12,3.15,7000.0,-12.921
